# Trading Strategy Backtester - EMA Crossover Strategy

This notebook demonstrates a complete backtesting workflow for an EMA crossover trading strategy with stop-loss and take-profit levels.

## Overview
- **Strategy**: EMA Crossover (9-period vs 15-period)
- **Timeframe**: 15-minute data
- **Assets**: EUR/USD and XAU/USD (Gold)
- **Risk Management**: Fixed Stop Loss and Take Profit
- **Analysis**: QuantStats performance reporting

## Data Requirements
- EURUSD data in `../data/EURUSD/` folder
- XAUUSD data in `../data/XAUUSD/` folder
- 15-minute timeframe CSV files

## Step 1: Import Required Libraries

Let's start by importing all the necessary libraries for our backtesting analysis.

In [ ]:
# Import core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import QuantStats for performance analysis
import quantstats as qs

# Set pandas display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Libraries imported successfully!")

## Step 2: Define the EMA Crossover Strategy Function

Now we'll create the core trading strategy function that implements:
- EMA crossover signals (9-period vs 15-period)
- Position entry on crossover
- Fixed stop-loss and take-profit exits
- Trade logging and tracking

In [ ]:
def perform_ema_backtest(data, short_window=9, long_window=15, sl_pct=0.01, tp_pct=0.03):
    """
    Performs an iterative backtest on an EMA crossover strategy with fixed SL/TP.
    
    Parameters:
    - data: DataFrame with OHLC data
    - short_window: Short EMA period (default: 9)
    - long_window: Long EMA period (default: 15)
    - sl_pct: Stop loss percentage (default: 1%)
    - tp_pct: Take profit percentage (default: 3%)
    
    Returns:
    - DataFrame with completed trades
    """
    
    # Calculate EMAs
    data['ema_short'] = data['Close'].ewm(span=short_window, adjust=False).mean()
    data['ema_long'] = data['Close'].ewm(span=long_window, adjust=False).mean()
    
    # Generate position signals (1 for long, -1 for short)
    data['position_signal'] = np.where(data['ema_short'] > data['ema_long'], 1, -1)
    
    # Detect crossover signals (changes from -1 to 1 = long entry, 1 to -1 = short entry)
    data['entry_signal'] = data['position_signal'].diff()
    
    # Initialize trade tracking
    in_position = False
    trades = []
    
    # Iterate through each candle
    for i in range(len(data)):
        # Check for long entry signal
        if not in_position and data['entry_signal'].iloc[i] == 2:  # Changed from -1 to 1
            in_position = True
            entry_price = data['Close'].iloc[i]
            stop_loss = entry_price * (1 - sl_pct)  # Stop loss below entry
            take_profit = entry_price * (1 + tp_pct)  # Take profit above entry
            
            # Record trade entry
            trades.append({
                'entry_date': data['DateTime'].iloc[i],
                'entry_price': entry_price,
                'trade_type': 'long',
                'stop_loss': stop_loss,
                'take_profit': take_profit,
                'exit_date': None,
                'exit_price': None,
                'pnl': 0,
                'exit_reason': None
            })
            
        # Check for exit conditions if in position
        elif in_position:
            exit_reason = None
            exit_price = 0
            
            # Check stop loss (if low touches stop loss)
            if data['Low'].iloc[i] <= stop_loss:
                exit_price = stop_loss
                exit_reason = "Stop Loss"
                
            # Check take profit (if high touches take profit)
            elif data['High'].iloc[i] >= take_profit:
                exit_price = take_profit
                exit_reason = "Take Profit"
                
            # Check opposing signal (EMA crossover in opposite direction)
            elif data['entry_signal'].iloc[i] == -2:  # Changed from 1 to -1
                exit_price = data['Close'].iloc[i]
                exit_reason = "Opposing Signal"
            
            # Execute exit if any condition met
            if exit_reason:
                trade = trades[-1]
                trade['exit_date'] = data['DateTime'].iloc[i]
                trade['exit_price'] = exit_price
                trade['pnl'] = exit_price - entry_price  # Calculate profit/loss
                trade['exit_reason'] = exit_reason
                in_position = False  # Reset position flag
    
    return pd.DataFrame(trades)

## Step 3: Create Performance Reporting Function

Now we'll create a comprehensive reporting function that uses QuantStats to generate detailed performance analytics including:
- Returns analysis
- Risk metrics
- Benchmark comparison
- Full tear sheet reports

In [ ]:
def generate_full_report(trades_df, original_data, title, initial_capital=10000.0):
    """
    Generates a full tear sheet report with robust handling for flat returns.
    
    Parameters:
    - trades_df: DataFrame with completed trades
    - original_data: Original price data for benchmark
    - title: Report title
    - initial_capital: Starting capital (default: $10,000)
    """
    
    # Check if we have any completed trades
    if trades_df.empty or trades_df['exit_date'].isna().all():
        print(f"\n--- {title} ---")
        print("No trades were completed, so no performance report can be generated.")
        return
    
    # Prepare trade data for analysis
    trades_df = trades_df.dropna(subset=['exit_date']).set_index('exit_date')
    
    # Calculate daily P&L from trades
    daily_pnl = trades_df['pnl'].resample('D').sum()
    
    # Create date range covering the entire period
    date_range = pd.date_range(
        start=original_data['DateTime'].min(), 
        end=original_data['DateTime'].max(), 
        freq='D'
    )
    
    # Build equity curve
    equity_curve = (initial_capital + daily_pnl.cumsum()).reindex(date_range, method='ffill')
    equity_curve.iloc[0] = initial_capital  # Set initial value
    equity_curve = equity_curve.ffill()  # Forward fill any missing values
    
    # Calculate strategy returns
    strategy_returns = equity_curve.pct_change(fill_method=None).fillna(0)
    strategy_returns.name = "Strategy"
    
    # Prepare benchmark data (buy & hold)
    benchmark_data = original_data.set_index('DateTime')['Close']
    benchmark_returns = benchmark_data.pct_change(fill_method=None).fillna(0)
    benchmark_returns.name = "Benchmark"
    
    print(f"\n--- {title} ---")
    print(f"Total Trades: {len(trades_df)}")
    print(f"Initial Capital: ${initial_capital:,.2f}")
    print(f"Final Capital: ${equity_curve.iloc[-1]:,.2f}")
    print(f"Total Return: {((equity_curve.iloc[-1] / initial_capital - 1) * 100):.2f}%")
    print("=" * 60)
    
    # Generate QuantStats report
    try:
        if strategy_returns.std() == 0:
            print("\nStrategy produced flat returns. Generating basic report.")
            qs.reports.basic(strategy_returns, benchmark=benchmark_returns)
        else:
            print("\nGenerating full performance report...")
            qs.reports.full(strategy_returns, benchmark=benchmark_returns)
    except Exception as e:
        print(f"Error generating QuantStats report: {e}")
        print("Falling back to basic statistics...")
        
        # Fallback: Basic statistics
        total_return = (equity_curve.iloc[-1] / initial_capital - 1) * 100
        print(f"Total Return: {total_return:.2f}%")
        
        if len(trades_df) > 0:
            win_rate = (trades_df['pnl'] > 0).mean() * 100
            avg_win = trades_df[trades_df['pnl'] > 0]['pnl'].mean()
            avg_loss = trades_df[trades_df['pnl'] < 0]['pnl'].mean()
            
            print(f"Win Rate: {win_rate:.1f}%")
            print(f"Average Win: ${avg_win:.2f}")
            print(f"Average Loss: ${avg_loss:.2f}")
            print(f"Profit Factor: {abs(avg_win / avg_loss) if avg_loss != 0 else float('inf'):.2f}")

## Step 4: Load EUR/USD 15-Minute Data

Let's load the EUR/USD 15-minute data from the data folder. Make sure the data file exists in the correct location.

In [ ]:
# Define data file paths
eurusd_data_path = '../data/EURUSD/EURUSD15.csv'
xauusd_data_path = '../data/XAUUSD/XAUUSD15.csv'

# Define column names for the CSV data
column_names = ['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume']

print("Loading EUR/USD 15-minute data...")

try:
    # Load EUR/USD data
    df_eurusd = pd.read_csv(eurusd_data_path, sep='\t', header=None, names=column_names)
    df_eurusd['DateTime'] = pd.to_datetime(df_eurusd['DateTime'])
    
    print(f"✅ EUR/USD data loaded successfully!")
    print(f"   - Records: {len(df_eurusd):,}")
    print(f"   - Date range: {df_eurusd['DateTime'].min()} to {df_eurusd['DateTime'].max()}")
    print(f"   - Price range: ${df_eurusd['Low'].min():.4f} - ${df_eurusd['High'].max():.4f}")
    
    # Display first few rows
    print("\nFirst 5 rows of EUR/USD data:")
    display(df_eurusd.head())
    
except FileNotFoundError:
    print(f"❌ Error: EUR/USD data file not found at {eurusd_data_path}")
    print("Please ensure the data file exists in the correct location.")
except Exception as e:
    print(f"❌ Error loading EUR/USD data: {e}")

## Step 5: Load XAU/USD 15-Minute Data

Now let's load the XAU/USD (Gold) 15-minute data.

In [ ]:
print("Loading XAU/USD 15-minute data...")

try:
    # Load XAU/USD data
    df_xauusd = pd.read_csv(xauusd_data_path, sep='\t', header=None, names=column_names)
    df_xauusd['DateTime'] = pd.to_datetime(df_xauusd['DateTime'])
    
    print(f"✅ XAU/USD data loaded successfully!")
    print(f"   - Records: {len(df_xauusd):,}")
    print(f"   - Date range: {df_xauusd['DateTime'].min()} to {df_xauusd['DateTime'].max()}")
    print(f"   - Price range: ${df_xauusd['Low'].min():.2f} - ${df_xauusd['High'].max():.2f}")
    
    # Display first few rows
    print("\nFirst 5 rows of XAU/USD data:")
    display(df_xauusd.head())
    
except FileNotFoundError:
    print(f"❌ Error: XAU/USD data file not found at {xauusd_data_path}")
    print("Please ensure the data file exists in the correct location.")
except Exception as e:
    print(f"❌ Error loading XAU/USD data: {e}")

## Step 6: Run EMA Backtest on EUR/USD

Let's execute the EMA crossover strategy on EUR/USD data with conservative risk parameters:
- Stop Loss: 0.5% (tight)
- Take Profit: 1.5% (moderate)

In [ ]:
print("🔄 Running EMA Crossover Backtest on EUR/USD...")
print("Parameters:")
print("- Short EMA: 9 periods")
print("- Long EMA: 15 periods")
print("- Stop Loss: 0.5%")
print("- Take Profit: 1.5%")
print("-" * 50)

# Run the backtest
eurusd_trades = perform_ema_backtest(
    df_eurusd.copy(), 
    short_window=9, 
    long_window=15, 
    sl_pct=0.005,  # 0.5% stop loss
    tp_pct=0.015   # 1.5% take profit
)

print(f"✅ EUR/USD backtest completed!")
print(f"Total trades generated: {len(eurusd_trades)}")

if not eurusd_trades.empty:
    print("\nTrade Summary:")
    print(eurusd_trades.describe())
    
    # Show sample trades
    print("\nSample trades:")
    display(eurusd_trades.head())
else:
    print("No trades were generated. Check your data and parameters.")

## Step 7: Generate Performance Report for EUR/USD

Now let's generate a comprehensive performance report for the EUR/USD strategy using QuantStats.

In [ ]:
# Generate full performance report for EUR/USD
generate_full_report(
    eurusd_trades, 
    df_eurusd, 
    title="EUR/USD EMA Crossover Strategy Performance",
    initial_capital=10000.0
)

## Step 8: Run EMA Backtest on XAU/USD

Now let's run the same strategy on XAU/USD (Gold) with slightly different risk parameters:
- Stop Loss: 1.0% (Gold is more volatile)
- Take Profit: 3.0% (Higher target for Gold)

In [ ]:
print("🔄 Running EMA Crossover Backtest on XAU/USD...")
print("Parameters:")
print("- Short EMA: 9 periods")
print("- Long EMA: 15 periods")
print("- Stop Loss: 1.0%")
print("- Take Profit: 3.0%")
print("-" * 50)

# Run the backtest
xauusd_trades = perform_ema_backtest(
    df_xauusd.copy(), 
    short_window=9, 
    long_window=15, 
    sl_pct=0.01,   # 1.0% stop loss (Gold is more volatile)
    tp_pct=0.03    # 3.0% take profit
)

print(f"✅ XAU/USD backtest completed!")
print(f"Total trades generated: {len(xauusd_trades)}")

if not xauusd_trades.empty:
    print("\nTrade Summary:")
    print(xauusd_trades.describe())
    
    # Show sample trades
    print("\nSample trades:")
    display(xauusd_trades.head())
else:
    print("No trades were generated. Check your data and parameters.")

## Step 9: Generate Performance Report for XAU/USD

Let's generate the performance report for the XAU/USD strategy.

In [ ]:
# Generate full performance report for XAU/USD
generate_full_report(
    xauusd_trades, 
    df_xauusd, 
    title="XAU/USD EMA Crossover Strategy Performance",
    initial_capital=10000.0
)

## Step 10: Strategy Comparison Summary

Let's create a summary comparison of both strategies' performance.

In [ ]:
print("📊 STRATEGY COMPARISON SUMMARY")
print("=" * 60)

# EUR/USD Summary
if not eurusd_trades.empty:
    eurusd_total_trades = len(eurusd_trades)
    eurusd_win_rate = (eurusd_trades['pnl'] > 0).mean() * 100
    eurusd_total_pnl = eurusd_trades['pnl'].sum()
    eurusd_avg_trade = eurusd_trades['pnl'].mean()
    
    print(f"EUR/USD Strategy:")
    print(f"  Total Trades: {eurusd_total_trades}")
    print(f"  Win Rate: {eurusd_win_rate:.1f}%")
    print(f"  Total P&L: ${eurusd_total_pnl:.2f}")
    print(f"  Average Trade: ${eurusd_avg_trade:.2f}")
else:
    print(f"EUR/USD Strategy: No trades generated")

print()

# XAU/USD Summary
if not xauusd_trades.empty:
    xauusd_total_trades = len(xauusd_trades)
    xauusd_win_rate = (xauusd_trades['pnl'] > 0).mean() * 100
    xauusd_total_pnl = xauusd_trades['pnl'].sum()
    xauusd_avg_trade = xauusd_trades['pnl'].mean()
    
    print(f"XAU/USD Strategy:")
    print(f"  Total Trades: {xauusd_total_trades}")
    print(f"  Win Rate: {xauusd_win_rate:.1f}%")
    print(f"  Total P&L: ${xauusd_total_pnl:.2f}")
    print(f"  Average Trade: ${xauusd_avg_trade:.2f}")
else:
    print(f"XAU/USD Strategy: No trades generated")

print("\n" + "=" * 60)
print("Backtesting completed! 📈")
print("\nNext steps:")
print("- Analyze the QuantStats reports above")
print("- Adjust strategy parameters if needed")
print("- Consider walk-forward optimization")
print("- Test on additional assets or timeframes")